In [96]:
import warnings
warnings.filterwarnings("ignore")

import geopandas as gpd
import pandas as pd

In [116]:
realgdf = gpd.read_file("../../../all_states_precincts_cname_neighbor_demo_OG_anomalous_fixNeighborVA.json")

In [117]:
path = "congressional 2016/"
file_name = "VA_Congressional2016_Cleaned_Mapped.csv"

In [118]:
cong16 = gpd.read_file(path+file_name)
cong16.drop(columns=["field_1"], inplace=True, axis=1)

In [119]:
realgdf = realgdf[realgdf["State"] == "va"]
c = list(realgdf.columns)
c.remove("County")
c.remove("Precinct #")
c.remove("CName")
realgdf.drop(columns=c, inplace=True, axis=1)
realgdf.head(2)

,County,Precinct #,CName
0,Accomack,0001,va-accomack-0001
1,Accomack,0002,va-accomack-0002


In [120]:
cong16.head(1)

,County,Precinct #,Republican,Democratic,Independent,Libertarian,Green,All Others,Total Votes Cast,dist_num,Precinct Name,geometry
0,Accomack County,0001,712,395,0,0,0,0,1107,2,Accomac,None


In [121]:
def normalizeCounty(gdf):
    for index, row in gdf.iterrows():
        splited = row["County"].split(" ")
        if splited[1] != "County":
            splited = ''.join(splited[:])
        else:
            splited = ''.join(splited[:-1])
        
        gdf.at[index, "County"] = splited
    return gdf
cong16 = normalizeCounty(cong16)

In [122]:
# create mapper
def createCountyPct(df):
    m = {}
    for index, row in df.iterrows():
        county = row["County"]
        if county not in m.keys():
            m[county] = {row["Precinct #"]: row["CName"]}
        else:
            if row["Precinct #"] in m[county].keys():
                # error catching step
                print(county, row["Precinct #"], row["CName"])
            else:
                m[county][row["Precinct #"]] = row["CName"]            
    return m

real = createCountyPct(realgdf)

In [123]:
# map the cname
for index, row in cong16.iterrows():
    cong16.at[index, "CName"] = real[row["County"]][row["Precinct #"]]

In [124]:
cong16.head()

,County,Precinct #,Republican,Democratic,Independent,Libertarian,Green,All Others,Total Votes Cast,dist_num,Precinct Name,geometry,CName
0,Accomack,0001,712,395,0,0,0,0,1107,2,Accomac,None,va-accomack-0001
1,Accomack,0002,595,126,0,0,0,2,723,2,Atlantic,None,va-accomack-0002
2,Accomack,0003,366,109,0,0,0,0,475,2,Bloxom,None,va-accomack-0003
3,Accomack,0004,413,170,0,0,0,1,584,2,Bobtown,None,va-accomack-0004
4,Accomack,0005,1220,397,0,0,0,2,1619,2,Chincoteague,None,va-accomack-0005


In [125]:
c = ["County", "Precinct #", "Precinct Name", "geometry"]
cong16.drop(columns=c, inplace=True, axis=1)
cong16.head()

,Republican,Democratic,Independent,Libertarian,Green,All Others,Total Votes Cast,dist_num,CName
0,712,395,0,0,0,0,1107,2,va-accomack-0001
1,595,126,0,0,0,2,723,2,va-accomack-0002
2,366,109,0,0,0,0,475,2,va-accomack-0003
3,413,170,0,0,0,1,584,2,va-accomack-0004
4,1220,397,0,0,0,2,1619,2,va-accomack-0005


In [126]:
cong16["Type"] = "CONG16"
cong16.head(1)

,Republican,Democratic,Independent,Libertarian,Green,All Others,Total Votes Cast,dist_num,CName,Type
0,712,395,0,0,0,0,1107,2,va-accomack-0001,CONG16


In [127]:
cong16.to_csv(file_name.split(".")[0] + "_Final.csv", index=False)